In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import random

import itertools

from cpd_utils import *
import changeforest

import time
import bisect

import pandas as pd

# ChangeForest

In [2]:
def generate_data_covariance(n, T, theta):
    p = theta.shape[1]
    y_train_joint = np.concatenate([
        np.random.multivariate_normal(mean = np.zeros(p), cov = theta[i], size = n[i]) 
        for i in range(T)], axis = 0)
#     y_train_joint = y_train.reshape((-1, p, p))
    nt = len(y_train_joint)
    
    return nt, y_train_joint

Signal too weak

In [21]:
T = 4
Delta = 500
n = np.array([Delta] * T)
cp_truth = np.cumsum(n)[:T-1]

p = 5
theta = np.zeros((T, p, p))
theta[0] = np.eye(p)

rho = 0.1
theta[1] = (1 - rho) * np.eye(p) + rho * np.ones((p, p))

rho = -0.1
theta[2] = (1 - rho) * np.eye(p) + rho * np.ones((p, p))

theta[3] = np.eye(p)

diff = np.zeros(T - 1)
for t in range(1, T):
    diff[t - 1] = np.sum(np.abs(theta[t] - theta[t - 1])**2)**0.5

nt, Y_train = generate_data_covariance(n, T, theta)
nt, Y_test = generate_data_covariance(n, T, theta)

In [22]:
theta.shape
Y_train.shape

(2000, 5)

In [23]:
result = changeforest.changeforest(Y_train, "random_forest", "bs")

In [24]:
result.split_points()

[]

In [25]:
result.segments

[OptimizerResult(start=0, stop=2000, best_split=1110, max_gain=-0.2455368062755241)]

Reasonable SNR

In [26]:
T = 4
Delta = 500
n = np.array([Delta] * T)
cp_truth = np.cumsum(n)[:T-1]

p = 5
theta = np.zeros((T, p, p))
theta[0] = np.eye(p)

rho = 0.3
theta[1] = np.eye(p)
for i in range(1, p):
    theta[1][i, i - 1] = rho
    theta[1][i - 1, i] = rho

rho = -0.3
theta[2] = np.eye(p)
for i in range(1, p):
    theta[2][i, i - 1] = rho
    theta[2][i - 1, i] = rho

theta[3] = np.eye(p)

diff = np.zeros(T - 1)
for t in range(1, T):
    diff[t - 1] = np.sum(np.abs(theta[t] - theta[t - 1])**2)**0.5

nt, Y_train = generate_data_covariance(n, T, theta)
nt, Y_test = generate_data_covariance(n, T, theta)

In [27]:
theta.shape
Y_train.shape

(2000, 5)

In [28]:
result = changeforest.changeforest(Y_train, "random_forest", "bs")

In [29]:
result.split_points()

[521, 1015, 1482, 1840]

In [11]:
result.segments

[OptimizerResult(start=0, stop=2000, best_split=20, max_gain=-22.48981242489963)]